In [ ]:

#here we have used data.csv and test.csv as train and have to predicted files

import pandas as pd
import seaborn as sb
from matplotlib import pyplot as plt
%matplotlib inline
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression

In [ ]:
#datacleaning

df=pd.read_csv("data.csv")

In [ ]:
df.GENDER.fillna('MISSING',inplace=True)
df.GENDER.replace('OTHER','MALE',inplace=True)

In [ ]:
#feature engineering

df=df.drop('No. of ATM Debit Transactions' ,axis=1)
df=df.drop('CUSTOMER_ID',axis=1)
df=df.drop(columns=['No. of Mobile Banking Debit Transactions','No. of Cheque Debit Transactions','Amount Withdrawn from ATM','Amount debited by Net Transactions','Amount debited by Cheque Transactions','Amount debited by Mobile Banking Transactions'])
df=df.drop('No. of Branch Cash Withdrawal Transactions'  ,axis=1)
df=df.drop('No. of Net Debit Transactions'  ,axis=1)
df=pd.get_dummies(df,columns=['GENDER','ACCOUNT TYPE','OCCUPATION'])

In [ ]:
#model training

y=df.TARGET
X=df.drop('TARGET',axis=1)

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1234,stratify=df.TARGET)
pipelines={
    'l1':make_pipeline(StandardScaler(),LogisticRegression(penalty='l1',random_state=123)),
    'l2':make_pipeline(StandardScaler(),LogisticRegression(penalty='l2',random_state=123)),
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier(random_state=123)),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier(random_state=123))
}
l1_hyperparameters={'logisticregression__C':[0.001,0.005,0.01,0.05,0.1,0.5,1,5,10,50,100,500,1000],}
l2_hyperparameters={'logisticregression__C':[0.001,0.005,0.01,0.05,0.1,0.5,1,5,10,50,100,500,1000],}
rf_hyperparameters={
    'randomforestclassifier__n_estimators':[100,200],
    'randomforestclassifier__max_features':['auto','sqrt',0.33],
    'randomforestclassifier__min_samples_leaf':[1,3,5,10]
}
gb_hyperparameters={
    'gradientboostingclassifier__n_estimators':[100,200],
    'gradientboostingclassifier__learning_rate':[0.05,0.1,0.2],
    'gradientboostingclassifier__max_depth':[1,3,5]
    
}

hyperparameters={'l1':l1_hyperparameters,'l2':l2_hyperparameters,'rf':rf_hyperparameters,'gb':gb_hyperparameters}
fitted_models={}

for name,pipeline in pipelines.items():
    model=GridSearchCV(pipeline,hyperparameters[name],cv=10,n_jobs=-1)
    model.fit(X_train,y_train)
    fitted_models[name]=model
    print(name)
for name,model in fitted_models.items():
    pred=model.predict_proba(X_test)
    pred=[p[1] for p in pred]
    
    print(name,roc_auc_score(y_test,pred))


In [ ]:
#selecting the winner
#the model with better score is picked here rf(random forest classifier)


#by cleaning the  file to be predicted and predict the values. 


In [ ]:
df1=pd.read_csv('test.csv')
df1.drop_duplicates(inplace=True)
df1.GENDER.fillna('MISSING',inplace=True)
df1.GENDER.replace('OTHER','MALE',inplace=True)
df1=df1.drop('No. of ATM Debit Transactions',axis=1)
df1=df1.drop(columns=['No. of Mobile Banking Debit Transactions','No. of Cheque Debit Transactions','Amount Withdrawn from ATM','Amount debited by Net Transactions','Amount debited by Cheque Transactions','Amount debited by Mobile Banking Transactions'])
df1=df1.drop('No. of Branch Cash Withdrawal Transactions'  ,axis=1)
df1=df1.drop('No. of Net Debit Transactions'  ,axis=1)

df1=pd.get_dummies(df1,columns=['GENDER','ACCOUNT TYPE','OCCUPATION'])



In [ ]:
y1=df1.TARGET
X1=df1.drop('TARGET',axis=1)

pred=fitted_models['gb'].predict(X1)

count=0
for i in range(len(pred)):
    if pred[i]==0:
        count=count+1
print(count)
df2=pd.DataFrame(pred)
df2.to_csv('target.csv')